In [2]:
print("Hello World")

Hello World


In [ ]:
import random
import pandas as pd
import math 
class YahtzeeSection:
    def __init__(self, name, is_set, score, number):
        self.name = name
        self.is_set = is_set
        self.score = score
        self.number = number

class Dice:
    def __init__(self, value):
        self.value = value
        self.locked = False

    def roll_unlocked_dice(self):
        if not self.locked:
            self.value = random.randint(1, 6)
class Game:
    def __init__(self):
        self.dice_set = [Dice(1), Dice(2), Dice(3), Dice(4), Dice(5)]
        self.sections = [
            YahtzeeSection("Ones", False, 0, 1),
            YahtzeeSection("Twos", False, 0, 2),
            YahtzeeSection("Threes", False, 0, 3),
            YahtzeeSection("Fours", False, 0, 4),
            YahtzeeSection("Fives", False, 0, 5),
            YahtzeeSection("Sixes", False, 0, 6),
        ]
        self.total_rounds = 6
        self.total_score = 0

    def roll_all_dice(self):
        for die in self.dice:
            die.roll_unlocked_dice()

    def lock_die(self, die_number):
        self.dice[die_number].locked = True

    def calculate_score(self):
        for section in self.sections:
            self.total_score += section.score

    def play_round(self):
        for i in range(3):
            self.roll_all_dice()
            print("Roll", i + 1)
            for die in self.dice:
                print(die.value)
            if i < 2:
                response = input("Which dice would you like to lock? (Enter the numbers separated by a space)")
                dice_to_lock = response.replace(',', ' ').split()
                print(dice_to_lock)
                for die in dice_to_lock:
                    self.lock_die(int(die) - 1)
        self.total_score = 0

    def play_game(self):
        for i in range(self.total_rounds):
            self.play_round()
            self.calculate_score()
            print("Total Score:", self.total_score)
    
    def probability_distribution_for_dice_set(self):
        for dice in self.dice_set:
            for i in range(6):
                filtered_dice_set = list(filter(lambda x: x.value == i + 1, self.dice_set))
                print(f"Number of {i + 1}s: {len(filtered_dice_set)}")
                remaining_dice = 5 - len(filtered_dice_set)
                print(f"Probability of rolling a {i + 1}: {}")
    
    def binomial_distribution(self, n, k, m):
        return (math.comb(n, k) * (p ** k) * ((1 - p) ** (n - k)))
    

    def buildagent(k, n, m):
        return math.factorial(2**k -1) / math.factorial(2**k + n) * math.factorial(2**k/m - 1)**m        
    

  



    
        

In [ ]:
game = Game()
game.play_game()

Roll 1
4
5
3
5
4


KeyboardInterrupt: Interrupted by user

: 

In [ ]:



# Generate five random integers between 1 and 6
random_numbers = [random.randint(1, 6) for _ in range(5)]
print(random_numbers)


[1, 1, 3, 6, 2]
